# 02 - Energy Analysis - Part 1
## Data preparation and EDA

### Data preparation

In [ ]:
# import libraries
import os
import numpy as np
import pandas as pd
import datetime
import math
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

# configs
%matplotlib inline
plt.rcParams["figure.figsize"] = 10, 6
pio.templates.default = "plotly_white"
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
# load my custom code
from myLib import data_analysis

In [ ]:
os.listdir("./data/raw")

#### Import energy data

In [ ]:
data_power = pd.read_csv("./data/raw/power-data-raw.csv", index_col=0)
data_power.head()

In [ ]:
data_power.index = data_power.index.astype("datetime64[ns]")

In [ ]:
data_analysis.plot_timeseries(
    df=data_power,
    y_traces=["Substation_total_kW", "Village_kW", "Mine_kW"],
    title="Energy data trends",
)

- There is some missing data in the Village_kW
- There appears to be an increase in energy consumption around May 2020

In [ ]:
# check for null values
data_power.isnull().mean()

- Impute missing values limiting it to 2019 values as consumption increased in 2020.

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
datetime_slice = (data_power.index >= datetime.datetime(2019, 1, 1)) & (
    data_power.index <= datetime.datetime(2019, 12, 31)
)
data_power.loc[datetime_slice, ["Village_kW"]] = imputer.fit_transform(
    data_power.loc[datetime_slice, ["Village_kW"]]
)

- Calculate the plant energy consumption = Total - Mine - Village
- Note values and converted from kW to kWh by multiplying by 0.5 (half-hourly data)
- Add filter to ensure values are greater than zero due to imputed values

In [ ]:
data_power["Plant_kWh"] = (
    data_power.Substation_total_kW - data_power.Village_kW - data_power.Mine_kW
) * 0.5
data_power["Plant_kWh"] = data_power[data_power.Plant_kWh > 0]

- Daily plant production is measure at 22:00, thus need to timeshift energy values to ensure daily totals are accurate

In [ ]:
data_power["ProdDateTime"] = data_power.index + datetime.timedelta(hours=1.5)
data_power["ProdDate"] = data_power.ProdDateTime.dt.date
data_power.head()

- Group data by daily totals

In [ ]:
data_energy = (
    data_power[["ProdDate", "Plant_kWh"]].groupby(by=["ProdDate"], as_index=False).sum()
)
data_energy.columns = ["DateOnly", "Actual-kWh"]
data_energy.DateOnly = data_energy.DateOnly.astype("datetime64[ns]")
data_energy.head()

#### Import production data
- data_prod (referred by Production1) are the values from the historian, available from Oct 2019
- data_prod_bi (referred by Production2) are the values from BI data that is calculated using weekly production hours, available for 2019

In [ ]:
data_prod = pd.read_csv("./data/raw/idx-historian-raw.csv", index_col=0)
data_prod.TimeStamp = data_prod.TimeStamp.astype("datetime64[ns]")
data_prod.head()

- Columns 3028 and 3301 are the tags for the two weightometers
- One is used as a backup, thus the average of these will be used

In [ ]:
data_prod_bi = pd.read_csv("./data/raw/bi-production-data-raw.csv")
data_prod_bi.DateOnly = data_prod_bi.DateOnly.astype("datetime64[ns]")
data_prod_bi.head()

### Combine production data

In [ ]:
data_prod["Production1"] = (data_prod["3028"] + data_prod["3301"]) / 2

In [ ]:
data_prod.columns

In [ ]:
data_prod.columns = ["DateOnly", "3028", "3301", "Production1"]

In [ ]:
data_combined = pd.merge(
    data_energy[["DateOnly", "Actual-kWh"]],
    data_prod[["DateOnly", "Production1"]],
    on="DateOnly",
)
data_combined.head()

In [ ]:
len(data_combined)

In [ ]:
data_combined_bi = pd.merge(
    data_combined[["DateOnly", "Actual-kWh", "Production1"]],
    data_prod_bi[["DateOnly", "Feed"]],
    on="DateOnly",
    how="outer",
)
data_combined_bi.head()

In [ ]:
data_combined_bi.tail()

In [ ]:
data_combined_bi.columns

In [ ]:
data_combined_bi.columns = ["DateOnly", "Actual_kWh", "Production1", "Production2"]

In [ ]:
data_analysis.plot_timeseries(
    df=data_combined_bi,
    y_traces=["Production1", "Production2"],
    title="Production data",
    x_trace="DateOnly",
    use_index=False,
)

- There are some production values in the BI dataset that are clear outliers as the plant capacity is 25k tons
- The BI data also seems to over estimate production based on more accurate readings from the historian

In [ ]:
# gap in Production1 data
data_combined_bi[
    (data_combined_bi.DateOnly > "2019-06-01")
    & (data_combined_bi.DateOnly < "2019-06-10")
]

In [ ]:
# gap in Production1 data
data_combined_bi[
    (data_combined_bi.DateOnly > "2019-09-07")
    & (data_combined_bi.DateOnly < "2019-10-08")
]

In [ ]:
filter0 = (data_combined_bi.DateOnly > "2019-06-06") & (
    data_combined_bi.DateOnly < "2019-09-10"
)
data_combined_bi["difference"] = (
    data_combined_bi.Production1 - data_combined_bi.Production2
)

In [ ]:
mean_difference = data_combined_bi.difference[filter0].mean()
print(f"Mean difference in production: {mean_difference:0.2f} tph")

In [ ]:
fig = px.box(
    data_combined_bi[filter0],
    y="difference",
    title="Difference between the two production datasets",
)
fig.show()

- Data from the BI dataset overestimates production
- Create an offset by calculating the mean and filtering using the whiskers of the box plot

In [ ]:
filter00 = (
    filter0
    & (data_combined_bi.difference > -7000)
    & (data_combined_bi.difference < 4000)
)
data_analysis.plot_timeseries(
    df=data_combined_bi[filter00],
    y_traces=["difference"],
    title="Compare difference between production data",
)

In [ ]:
mean_difference = data_combined_bi.difference[filter00].mean()
print(f"Mean difference in production: {mean_difference:0.2f} tph")

- Create a new column, called Feed, with the adjusted production values included

In [ ]:
data_combined_bi["Feed"] = data_combined_bi.Production1

In [ ]:
filter1 = (data_combined_bi.DateOnly < "2019-06-06") & (
    data_combined_bi.Production2 < 25000
)
data_combined_bi.Feed[filter1] = data_combined_bi.Production2[filter1] + mean_difference

In [ ]:
filter2 = (
    (data_combined_bi.DateOnly > "2019-09-09")
    & (data_combined_bi.DateOnly < "2019-10-01")
    & (data_combined_bi.Production2 < 25000)
)
data_combined_bi.Feed[filter2] = data_combined_bi.Production2[filter2] + mean_difference
data_combined_bi.Feed[data_combined_bi.Feed < 0] = 0

In [ ]:
data_analysis.plot_timeseries(
    df=data_combined_bi,
    y_traces=["Production1", "Production2", "Feed"],
    title="Production data",
    x_trace="DateOnly",
    use_index=False,
)

In [ ]:
data_combined_bi.drop(["difference"], axis=1, inplace=True)
data_combined_bi.to_csv("./data/raw/combined-data-raw.csv")

In [ ]:
data_combined_bi.head()

## Prepare for modelling and evaluation

In [ ]:
raw_data = pd.read_csv("./data/raw/combined-data-raw.csv", index_col=0)
raw_data.head()

In [ ]:
raw_data.tail()

In [ ]:
raw_data.columns

In [ ]:
raw_data.columns = ["ProdDate", "Actual_kWh", "Production1", "Production2", "Feed"]

In [ ]:
raw_data.info()

In [ ]:
# set date formats
raw_data.ProdDate = raw_data.ProdDate.astype("datetime64[ns]")
raw_data["Weekday"] = raw_data.ProdDate.dt.day_name()

In [ ]:
raw_data.head()

In [ ]:
raw_data.describe()

- The baseline period for analysis will be the first 9 months, before APC was implemented
- The reporting period will be from October 2019 where APC was utilised and will be compared with the baseline period

In [ ]:
# set period labels
raw_data["BRP"] = ""

# baseline period data
raw_data.BRP[
    (raw_data.ProdDate >= "2019-01-01") & (raw_data.ProdDate < "2019-10-01")
] = "BL"

# reporting period data
raw_data.BRP[(raw_data.ProdDate >= "2019-10-01")] = "RP"

In [ ]:
# split data set after clean up
interim_data_bl = raw_data[raw_data["BRP"] == "BL"]
interim_data_rp = raw_data[raw_data["BRP"] == "RP"]
print(len(interim_data_bl.ProdDate))
print(len(interim_data_rp.ProdDate))

In [ ]:
interim_data_path = "./data/interim/"
interim_data_bl.to_csv(interim_data_path + "interim_data_bl.csv")
interim_data_rp.to_csv(interim_data_path + "interim_data_rp.csv")

## Exploratory data analysis

In [ ]:
os.listdir("./data/interim")

In [ ]:
data_bl = pd.read_csv("./data/interim/interim_data_bl.csv", index_col=0)
data_bl.head()

In [ ]:
data_bl.info()

In [ ]:
data_bl.ProdDate = data_bl.ProdDate.astype("datetime64[ns]")

In [ ]:
data_bl = data_bl.replace([np.inf, -np.inf], np.nan)
data_bl.fillna(0, inplace=True)

In [ ]:
data_bl.describe()

In [ ]:
len(data_bl.ProdDate)

- Check for stale Feed values, i.e. values that got stuck from the historian and where interpolated incorrectly

In [ ]:
data_bl["check_feed"] = data_bl.Feed.eq(data_bl.Feed.shift())
data_bl = data_bl[data_bl["check_feed"] == False]

In [ ]:
len(data_bl.ProdDate)

In [ ]:
data_bl.columns

In [ ]:
# plot histograms for the labels below
feat_cols = ["Feed"]
feat_labs_cols = feat_cols + ["Actual_kWh"]
data_analysis.plot_hist(feat_labs_cols, data_bl)

- Zero values stand out, typically when the plant was not running
- Let's look at the daily average energy profiles

In [ ]:
# plot historgram per weekday
x1 = data_bl.Actual_kWh[data_bl.Weekday == "Monday"]
x2 = data_bl.Actual_kWh[data_bl.Weekday == "Tuesday"]
x3 = data_bl.Actual_kWh[data_bl.Weekday == "Wednesday"]
x4 = data_bl.Actual_kWh[data_bl.Weekday == "Thursday"]
x5 = data_bl.Actual_kWh[data_bl.Weekday == "Friday"]
x6 = data_bl.Actual_kWh[data_bl.Weekday == "Saturday"]
x7 = data_bl.Actual_kWh[data_bl.Weekday == "Sunday"]
hist_data = [x1, x2, x3, x4, x5, x6, x7]
group_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

fig = ff.create_distplot(
    hist_data,
    group_labels,
    bin_size=data_analysis.hist_bin_width_fd(data_bl.Actual_kWh),
    show_rug=False,
    curve_type="kde",
)

fig["layout"].update(title="Histogram for energy consumption per weekday")
fig.show()

- The plant does not normally operate on Sundays, thus a non-productive period
- Thurdays also stands out and this is when weekly maintenance occurs during the day shift

In [ ]:
# Filter data for the productive period, both energy and production
data_bl = data_bl[data_bl.Actual_kWh > 55000]
data_bl = data_bl[data_bl.Feed > 200]

In [ ]:
# compute the correlation matrix
data_bl_corr = data_bl[feat_labs_cols].corr()

# set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

# generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    data_bl_corr,
    cmap=cmap,
    vmax=0.3,
    center=0,
    annot=True,
    robust=True,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)
plt.show()

In [ ]:
print(feat_cols)

In [ ]:
# remove columns
feat_cols = ["Feed"]
print(feat_cols)

feat_labs_cols = feat_cols + ["Actual_kWh"]
print(feat_labs_cols)

In [ ]:
# scatter plot
import colorlover as cl

scl = cl.scales["9"]["seq"]["Blues"]
colorscale = [[float(i) / float(len(scl) - 1), scl[i]] for i in range(len(scl))]

trace = go.Scatter(
    x=data_bl.Feed,
    y=data_bl.Actual_kWh,
    text=data_bl.index,
    mode="markers+text",
    textposition="top center",
    hoverinfo="text",
    marker=dict(opacity=0.5, sizemin=5, sizemode="area"),
)
trace_c = go.Histogram2dContour(
    x=data_bl.Feed,
    y=data_bl.Actual_kWh,
    ncontours=5,
    colorscale=colorscale,
    showscale=False,
    opacity=0.3,
)
data = [trace, trace_c]
layout = go.Layout(title="Scatter plot")
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# locate rows that appear to be outliers
data_bl.loc[[244, 211, 162, 251, 218, 159, 56, 130, 74, 55, 28]].sort_values(
    by="ProdDate"
)

In [ ]:
# drop
data_bl.drop([244, 211, 162, 251, 218, 159, 56, 130, 74, 55, 28], inplace=True)

In [ ]:
sns.lmplot(
    data=data_bl,
    x="Feed",
    y="Actual_kWh",
    lowess=True,
    line_kws={"color": "red"},
    aspect=1.6,
)
sns.kdeplot(data=data_bl, x="Feed", y="Actual_kWh")
plt.show()

In [ ]:
data_bl.to_csv("./data/processed/processed_data_bl.csv")